In [54]:
load("clean_data_nov172018.RData")
df.base <- df.clean



In [55]:
# Get y 
regress.df <- df.clean
cost <- df.clean$cost
# Add ship_year and ship_month
df.base$ship_year <- as.numeric(format(as.Date(df.clean$ship_time), '%Y'))
df.base$ship_month <- as.numeric(format(as.Date(df.clean$ship_time), '%m'))

# Add long-term fixed effect
regress.df$trend <- (df.base$ship_year - 2016)*52 + df.base$ship_month

# Add month effect, vsl_company, total_weight
regress.df$ship_month <- as.factor(df.base$ship_month)
regress.df$port <- df.base$port
regress.df$total_weight <- df.base$total_weight
regress.df$vsl_type <- df.base$vsl_type



In [56]:
regress.df <- subset(regress.df,select=-dta_collection_time)

In [57]:
regress.df <- subset(regress.df,select=-contract_time)

In [58]:
regress.df <- subset(regress.df,select=-bid_time)

In [59]:
regress.df <- subset(regress.df,select=-vsl_company)

In [60]:
regress.df <- subset(regress.df,select=-vsl_name)

In [61]:
regress.df <- subset(regress.df,select=-port_type_value)
regress.df <- subset(regress.df,select=-port_price_diff)
regress.df <- subset(regress.df,select=-income)
regress.df <- subset(regress.df,select=-bang_dian_cost)
regress.df <- subset(regress.df,select=-chao_qi_cost)
regress.df <- subset(regress.df,select=-kua_gang_cost)



In [64]:
cost <- regress.df$cost[1:409]

In [65]:
# Divide by training and testing
regress.train <- regress.df[1:409,]
regress.test <- regress.df[410:531,]


model1 <- lm(cost ~., data = regress.train)

# Summaize the result
tmp <- c(mean(abs(model1$residuals)), mean(abs(model1$residuals)/regress.train$cost),summary(model1)$r.squared)

# Prediction
regress.test$port[regress.test$port == 'JiaoJiang'] = 'JiaoJiangSSZD'
result_test_raw <- data.frame(true = regress.test$cost, pred = predict(model1, regress.test))
result_test_raw$error <- result_test_raw$true - result_test_raw$pred
tmp2 <- c(mean(abs(result_test_raw$error)), mean(abs(result_test_raw$error)/result_test_raw$true), NA)
summary_result_model1 <- data.frame(train = tmp, test = tmp2)
rownames(summary_result_model1) <- c("APE", "MAPE", "R-Squared")



Warning message in predict.lm(model1, regress.test):
"prediction from a rank-deficient fit may be misleading"

In [66]:
result_test_raw <- data.frame(true = regress.test$cost, pred = predict(model1, regress.test))


Warning message in predict.lm(model1, regress.test):
"prediction from a rank-deficient fit may be misleading"

In [78]:
# Plot the prediction and true values
library('data.table')
library('ggplot2')
regress.tmp <- regress.df
regress.tmp$port[regress.tmp$port == 'JiaoJiang'] = 'JiaoJiangSSZD'
result_tmp <- data.frame(true = regress.tmp$cost, pred = predict(model1, regress.tmp), ship_time = df.base$ship_time)
plot.df <- melt(result_tmp, id.vars = "ship_time")



## Sanity check
#tmp <- regress.df %>% select(-1)
#tmp2 <- unique(tmp)
## Passed! No repetition of rows

Warning message in predict.lm(model1, regress.tmp):
"prediction from a rank-deficient fit may be misleading"

In [79]:
p <- ggplot2(plot.df, aes(x = ship_time, y = value, group = variable, color = variable)) + geom_line() + geom_point() + 
     geom_vline(xintercept = df.base$bid_time[410])
p

ERROR: Error in ggplot2(plot.df, aes(x = ship_time, y = value, group = variable, : 没有"ggplot2"这个函数
